In [1]:
import numpy as np

In [2]:
%run ../load_data.py
%run ../dir_utils.py

Using TensorFlow backend.


# Test generator function

In [3]:
batch_size=128
#filepath='/home/zhampel/insight/data/mnist/pngs'
trainpath='/home/zhampel/insight/data/cifar-10-images'
data_dir_struct = DataDirStruct(trainpath)
train_gen, valid_gen = train_img_generator(dir_struct=data_dir_struct, \
                                           batch_size=batch_size, \
                                           val_split=0.2)

Classes: 10, Image Dims: (32, 32)
Found 40000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [4]:
test_gen = test_img_generator(dir_struct=data_dir_struct, \
                              batch_size=batch_size)
print(test_gen.image_shape)
print(test_gen.num_classes)
print(test_gen.batch_size)

Classes: 10, Image Dims: (32, 32)
Found 10000 images belonging to 10 classes.
(32, 32, 3)
10
128


In [5]:
import keras
from keras.layers import Dense, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.utils import np_utils


np.random.seed(1671)
np.set_printoptions(threshold='nan')

In [6]:
class History(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.val_acc = []
        self.loss = []
        self.val_loss = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

In [7]:
from keras.layers.convolutional import Convolution2D
from keras import backend as K
import tensorflow as tf

In [8]:
permutation = [[1, 0], [0, 0], [0, 1], [2, 0], [1, 1], [0, 2], [2, 1], [2, 2], [1, 2]]


def shift_rotate(w, shift=1):
    shape = w.get_shape()
    for i in range(shift):
        w = tf.reshape(tf.gather_nd(w, permutation), shape)
    return w

In [14]:
w = np.array(
    [
        [
            [
                [1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]
            ],
            [
                [11, 22, 33],
                [44, 55, 66],
                [77, 88, 99]
            ],
            [
                [111, 222, 333],
                [444, 555, 666],
                [777, 888, 999]
            ]
        ],
        [
            [
                [1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]
            ],
            [
                [11, 22, 33],
                [44, 55, 66],
                [77, 88, 99]
            ],
            [
                [111, 222, 333],
                [444, 555, 666],
                [777, 888, 999]
            ]
        ]
    ]
)

w = np.moveaxis(w, (2, 3), (0, 1))

sess = tf.InteractiveSession()
# shape: (row, col, inp_nb, out_nb) -> (3, 3, 2, 3)
w = tf.constant(w)

w1 = shift_rotate(w, 4)
print(w1.eval())

/home/zhampel/py-venv/insight/lib/python3.5/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


TypeError: eval() arg 1 must be a string, bytes or code object

In [ ]:
class Convolution2D_4(Convolution2D):
    def call(self, x, mask=None):
        w = self.kernel
        w_rot = [w]
        for i in range(3):
            w = shift_rotate(w, shift=2)
            w_rot.append(w)

        outputs = tf.stack([K.conv2d(x, w_i, strides=self.strides,
                                     padding=self.padding,
                                     data_format=self.data_format) for w_i in w_rot])

        output = K.max(outputs, 0)

        if self.bias:
            if self.data_format == 'th':
                output += K.reshape(self.b, (1, self.nb_filter, 1, 1))
            elif self.data_format == 'tf':
                output += K.reshape(self.b, (1, 1, 1, self.nb_filter))
            else:
                raise ValueError('Invalid data_format:', self.data_format)
        output = self.activation(output)
        return output

In [ ]:
# Grab input shape and class numbers from generator
# Specifics
batch_size = test_gen.batch_size
epochs = 100
input_shape = train_gen.image_shape
num_classes = train_gen.num_classes
# History
history = History()
# Model
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Convolution2D_4(64, kernel_size=(3, 3),
                          use_bias=False, 
                          padding='same', 
                          activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
# Run the training
model.fit_generator(train_gen,
                    steps_per_epoch=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=valid_gen,
                    validation_steps=10,
                    class_weight=None,
                    callbacks=[history])